In [288]:
##set up the environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [289]:
#load Data
dataset = pd.read_json('../tweets/formatted-tweets.txt')
dataset.shape
dataset.head()

authors = pd.read_json('../tweets/formatted-authors.txt')
authors.shape

(26728, 13)

In [290]:
#filter and merge
hurtfulTweets = dataset[dataset["score"]>=1]
y = hurtfulTweets[~pd.isnull(hurtfulTweets["reply_to_pseudo"]) | ~pd.isnull(hurtfulTweets["user_mentions_pseudo"])]
y.shape
x = authors.drop_duplicates('author_pseudo')
x.shape
attacks = y.merge(x, left_on='author_id', right_on='author_id')
attacks = attacks.loc[:,['emotion','hurtClass','reply_to_pseudo','score','user_mentions_pseudo','author_pseudo']]

In [291]:
#loop attacks to quantify users
users = x.loc[:,["author_pseudo","total_send","total_received"]]
users["total_send"] = 0
users["total_received"] = 0

for attack in attacks[pd.isnull(attacks['user_mentions_pseudo'])].iterrows():
    #attacks
    users.loc[users["author_pseudo"]==attack[1]['author_pseudo'],"total_send"] += attack[1]['score']
    #directly received
    users.loc[users["author_pseudo"]==attack[1]['reply_to_pseudo'],"total_received"] += attack[1]['score']
  
for attack in attacks[~pd.isnull(attacks['user_mentions_pseudo'])].iterrows():
    #attacks
    users.loc[users["author_pseudo"]==attack[1]['author_pseudo'],"total_send"] += attack[1]['score']
    #directly received
    users.loc[users["author_pseudo"]==attack[1]['reply_to_pseudo'],"total_received"] += attack[1]['score']
    #mentioned
    for mention in attack[1]['user_mentions_pseudo']: 
        users.loc[users["author_pseudo"]==mention,"total_received"] += attack[1]['score']


In [292]:
users.head()

,author_pseudo,total_send,total_received
0,zaphodgjd,10,0
1,SleezyAlan,0,0
2,lindasue0818,4,0
3,falizeastudillo,6,0
4,Kavithamadhusu1,0,0


In [293]:
users.to_csv('users.csv')

In [311]:
attacks = y.merge(x, left_on='author_id', right_on='author_id')
attacks[attacks['author_pseudo']=='GodofGreen2'].loc[:,['tweet_content','emotion','hurtClass']].to_csv('GodofGreen2.csv')

In [287]:
dataset.loc[600:800,['tweet_content','hurtClass','class']].to_csv('to_tag1.csv')
dataset.loc[1000:1200,['tweet_content','hurtClass','class']].to_csv('to_tag2.csv')
dataset.loc[1300:1500,['tweet_content','hurtClass','class']].to_csv('to_tag3.csv')
dataset.loc[1700:1900,['tweet_content','hurtClass','class']].to_csv('to_tag4.csv')
dataset.loc[2000:2200,['tweet_content','hurtClass','class']].to_csv('to_tag5.csv')
dataset.loc[2300:2500,['tweet_content','hurtClass','class']].to_csv('to_tag6.csv')
dataset.loc[2700:2900,['tweet_content','hurtClass','class']].to_csv('to_tag7.csv')
dataset.loc[3000:3200,['tweet_content','hurtClass','class']].to_csv('to_tag8.csv')
dataset.loc[3300:3500,['tweet_content','hurtClass','class']].to_csv('to_tag9.csv')
dataset.loc[3700:3900,['tweet_content','hurtClass','class']].to_csv('to_tag10.csv')

In [308]:
#test accuracy
tagged = pd.read_csv('to_tag3.csv')
tagged = tagged[~pd.isnull(tagged['class'])]
tagged['class'] = tagged['class'].astype(int)

accuracy_score(tagged['class'],tagged['hurtClass'])
f1_score(tagged['class'],tagged['hurtClass'],average='weighted')
precision_recall_fscore_support(tagged['class'],tagged['hurtClass'], labels=[1,2,3])

(array([ 0.52083333,  0.94230769,  0.16666667]),
 array([ 0.83333333,  0.6125    ,  0.8       ]),
 array([ 0.64102564,  0.74242424,  0.27586207]),
 array([ 30, 160,  10]))

In [309]:
#add tagged data to tagged archive
tagged.to_csv('tagged_data.csv', mode='a',header=False) 

In [313]:
attacks.dtypes

author_id                           int64
emotion                            object
favorite_count                      int64
favorited_x                          bool
hashtags                           object
hurtClass                           int64
inserted_in_es_at_x        datetime64[ns]
is_a_retweet                         bool
reply_to_pseudo                    object
reply_to_tweet                    float64
reply_to_user_id                  float64
retweet_count                       int64
score                               int64
tweet_content                      object
tweet_coordinates                  object
tweet_created_at           datetime64[ns]
tweet_id                            int64
tweet_lang                         object
urls                               object
user_mentions_id                   object
user_mentions_pseudo               object
author_account_creation            object
author_description                 object
author_followers_count            